# Definindo sistema


In [1]:
import numpy as np
import torch

from lib.physics import simulate
from lib.plots import plot_tanks

t = np.linspace(0, 300, 300)
t_tensor = torch.tensor(t, dtype=torch.float32, requires_grad=True).unsqueeze(1)
y0 = np.array([0, 0])
sol = simulate(y0, t_eval=t)

# Add noise
h1_exp = sol[0] + np.random.normal(0, 1, len(t)) * 0.1
h2_exp = sol[1] + np.random.normal(0, 1, len(t)) * 0.1

plot_tanks(
    t, [h1_exp, h2_exp], ["h1 (exp)", "h2 (exp)"], scatter=2, filename="exp_tanks"
)

# from lib.physics import F
# plt.figure(figsize=(10, 4), layout="constrained")
# plt.plot(t, F(t), label="F")
# plt.show()


# Definindo rede neural


In [2]:
from lib.BaseModel import BaseModel


def getNewModel():
    return BaseModel(
        max_input=float(torch.max(t_tensor)),
        max_output=float(torch.max(h1_exp)),
    )


In [3]:
from lib.physics import edo_torch
from lib.utils import dydx, mean_square

h1_exp = torch.tensor(h1_exp, dtype=torch.float32)
h2_exp = torch.tensor(h2_exp, dtype=torch.float32)


def loss_fn(model, t: torch.Tensor):
    # Loss das EDOs
    Y_pred = model(t)
    h1_pred, h2_pred = Y_pred[:, 0], Y_pred[:, 1]

    dh1dt_pinn, dh2dt_pinn = dydx(t, h1_pred), dydx(t, h2_pred)
    dh1dt_edo, dh2dt_edo = edo_torch(t, [h1_pred, h2_pred])

    loss_EDO1 = mean_square(dh1dt_pinn - dh1dt_edo)
    loss_EDO2 = mean_square(dh2dt_pinn - dh2dt_edo)

    # Loss das condições iniciais
    t0 = torch.tensor([[0.0]], requires_grad=True)
    Y0 = model(t0)
    h1_0, h2_0 = Y0[:, 0], Y0[:, 1]

    loss_ic1 = mean_square(h1_0 - y0[0])
    loss_ic2 = mean_square(h2_0 - y0[1])

    # Loss dos dados
    loss_data_h1 = mean_square(h1_pred - h1_exp)
    loss_data_h2 = mean_square(h2_pred - h2_exp)

    # Loss total
    loss_total = (
        loss_EDO1 + loss_EDO2 + loss_data_h1 + loss_data_h2 + loss_ic1 + loss_ic2
    )

    return loss_total


# Testando métodos


In [4]:
n_execuções = 5
target_loss = 0.1
trials = 100

Adam_study_path = "../results/Adam-studies.hkl"
Adam_results_path = "../results/Adam-speeds.hkl"
Adam_model_path = "../results/Adam-model.pt"

GA_study_path = "../results/GA-studies.hkl"
GA_results_path = "../results/GA-speeds.hkl"
GA_model_path = "../results/GA-model.pt"


def count_fails(losses):
    return np.sum(np.array(losses) > target_loss)


## Adam


### Otimizando hiperparametros


In [5]:
import optuna

from lib.optuna import study
from lib.utils import train


def objective(trial: optuna.Trial):
    torch.manual_seed(42)
    test_model = getNewModel()

    lr = trial.suggest_float("lr", 1e-15, 1)
    beta1 = trial.suggest_float("beta1", 1e-10, 1)
    beta2 = trial.suggest_float("beta2", 1e-10, 1)

    optimizer = torch.optim.Adam(test_model.parameters(), lr=lr, betas=(beta1, beta2))

    return train(test_model, loss_fn, optimizer, 1000, t_tensor)


best_Adam_params = study(objective, trials, "Adam-study", Adam_study_path)


[I 2025-01-13 01:01:06,382] A new study created in memory with name: Adam-study


Otimização realizada anteriormente. Recuperando valores...
Imprimindo resultado:
  Valor do Loss: 0.0197629164904356
  hiperparametros:
    lr: 0.004589063269254406
    beta1: 0.9564560540424447
    beta2: 0.8804782307261265


### Medindo desempenho


In [6]:
from lib.test_speed import test_train_speed


def Adam_train(model, seed):
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=best_Adam_params["lr"],
        betas=(best_Adam_params["beta1"], best_Adam_params["beta2"]),
    )
    loss_value = train(model, loss_fn, optimizer, 5000, t_tensor, target_loss)
    return loss_value


Adam_times, Adam_losses, Adam_model = test_train_speed(
    Adam_train, getNewModel, n_execuções, Adam_results_path, Adam_model_path
)

print(f"Média do tempo (Adam): {np.mean(Adam_times):.3f}s")
print("Tentativas falhadas:", count_fails(Adam_losses))


Recuperando testes anteriores
Média do tempo (Adam): 1.325s
Tentativas falhadas: 0


In [7]:
@torch.inference_mode()
def test_model(model):
    y = model(t_tensor)
    return [y[:, 0], y[:, 1]]


pinn_h1, pinn_h2 = test_model(Adam_model)

# Gráfico
plot_tanks(
    t,
    (h1_exp, h2_exp, pinn_h1, pinn_h2),
    ["h1 (exp)", "h2 (exp)", "h1 (PINN Adam)", "h2 (PINN Adam)"],
    scatter=2,
    filename="adam_tanks",
)


## Algoritmo Genético


In [8]:
import pygad
from pygad.torchga import torchga

GA_model = getNewModel()


def fitness_func(ga_instance, solution, solution_idx):
    model_weights_dict = torchga.model_weights_as_dict(
        model=GA_model, weights_vector=solution
    )
    GA_model.load_state_dict(model_weights_dict)

    GA_model.eval()
    loss = loss_fn(GA_model, t_tensor)

    # Quanto menor o loss, maior o fitness
    return -loss.item()


### Otimizando hiperparametros


In [9]:
def objective(trial: optuna.Trial):
    torch.manual_seed(42)
    test_model = getNewModel()

    parent_selection_type = trial.suggest_categorical(
        "parent_selection_type", ["sss", "rws", "sus", "rank", "random", "tournament"]
    )
    keep_elitism = trial.suggest_int("keep_elitism", 0, 10)
    num_parents_mating = trial.suggest_int("num_parents_mating", 2, 10)

    crossover_type = trial.suggest_categorical(
        "crossover_type", ["single_point", "two_points", "uniform", "scattered", None]
    )
    crossover_probability = trial.suggest_float("crossover_probability", 0, 1)

    mutation_type = trial.suggest_categorical(
        "mutation_type", ["random", "swap", "inversion", "scramble", "adaptive", None]
    )
    mutation_probability = trial.suggest_float("mutation_probability", 0, 1)

    # Configura o TorchGA para criar populações baseadas no modelo
    torch_ga = torchga.TorchGA(model=test_model, num_solutions=50)

    # Configura o algoritmo genético
    ga_instance = pygad.GA(
        # Configurações
        initial_population=torch_ga.population_weights,  # População inicial
        fitness_func=fitness_func,  # Função de aptidão
        num_generations=80,  # Número de gerações
        random_seed=42,
        init_range_low=-4,
        init_range_high=4,
        # Parâmetros para otimizar
        parent_selection_type=parent_selection_type,
        keep_elitism=keep_elitism,
        num_parents_mating=num_parents_mating,
        crossover_type=crossover_type,  # type: ignore
        crossover_probability=crossover_probability,
        mutation_type=mutation_type,  # type: ignore
        mutation_probability=(
            mutation_probability if mutation_type != "adaptive" else [0.8, 0.1]
        ),
    )

    # Executa o algoritmo genético
    ga_instance.run()

    _, best_solution_fitness, _ = ga_instance.best_solution()

    return float(best_solution_fitness)


best_GA_params = study(
    objective, trials, "GA-study", GA_study_path, study_direction="maximize"
)


[I 2025-01-13 01:01:06,843] A new study created in memory with name: GA-study


Otimização realizada anteriormente. Recuperando valores...
Imprimindo resultado:
  Valor do Loss: -0.10038375109434128
  hiperparametros:
    parent_selection_type: sss
    keep_elitism: 4
    num_parents_mating: 6
    crossover_type: single_point
    crossover_probability: 0.10600715915754366
    mutation_type: adaptive
    mutation_probability: 0.44684389236973643


### Medindo desempenho


In [10]:
def on_generation(ga_instance):
    if ga_instance.best_solution()[1] > -target_loss:
        print("Chegou no loss alvo antes de terminar as gerações!")
        return "stop"

    return None


if best_GA_params["mutation_type"] == "adaptive":
    best_GA_params["mutation_probability"] = [0.8, 0.1]


def GA_train(model, seed):
    torch_ga = torchga.TorchGA(model=model, num_solutions=200)
    ga_instance = pygad.GA(
        initial_population=torch_ga.population_weights,
        fitness_func=fitness_func,
        random_seed=seed,
        num_generations=500,
        init_range_low=-4,
        init_range_high=4,
        on_generation=on_generation,
        **best_GA_params,
    )
    ga_instance.run()
    best_solution, best_solution_fitness, _ = ga_instance.best_solution()

    model_weights_dict = torchga.model_weights_as_dict(
        model=model, weights_vector=best_solution
    )
    model.load_state_dict(model_weights_dict)

    return -best_solution_fitness


GA_times, GA_losses, GA_model = test_train_speed(
    GA_train, getNewModel, n_execuções, GA_results_path, GA_model_path
)

print(f"Média do tempo (GA): {np.mean(GA_times):.3f}s")
print("Tentativas falhadas:", count_fails(GA_losses))


Recuperando testes anteriores
Média do tempo (GA): 334.755s
Tentativas falhadas: 0


In [11]:
pinn_h1, pinn_h2 = test_model(GA_model)

# Gráfico
plot_tanks(
    t,
    (h1_exp, h2_exp, pinn_h1, pinn_h2),
    ["h1 (exp)", "h2 (exp)", "h1 (PINN GA)", "h2 (PINN GA)"],
    scatter=2,
    filename="ga_tanks",
)


In [12]:
for nome, param in GA_model.named_parameters():
    print(f"Nome: {nome}")
    print(f"Valor: {param}")


Nome: hidden_layer.0.weight
Valor: Parameter containing:
tensor([[-0.3663],
        [-0.7181],
        [-0.6447],
        [ 0.1716],
        [-1.1295],
        [-1.3212],
        [ 1.4641],
        [ 0.4382]], requires_grad=True)
Nome: hidden_layer.0.bias
Valor: Parameter containing:
tensor([ 0.7772,  1.1267, -1.5652, -1.8301,  1.5276, -0.8362, -0.7261,  0.0630],
       requires_grad=True)
Nome: hidden_layer.2.weight
Valor: Parameter containing:
tensor([[ 1.2457,  0.3030,  0.5481, -0.9717, -0.2227,  0.4674, -1.6770, -0.6474],
        [-0.0264, -1.3338,  0.6143, -0.0054, -0.8995, -0.0269,  0.0870, -0.1910],
        [ 0.1211,  0.3470,  0.7707, -1.1534, -0.3529,  1.8391,  0.4926,  0.6483],
        [ 0.1665, -0.1331,  1.1962,  0.3179, -0.2468,  1.0219, -0.2354, -0.0478],
        [ 0.8683,  0.8412,  0.3364, -0.0269, -1.2083, -0.4037,  0.2675,  0.7069],
        [ 0.4684,  0.7118, -0.4320,  1.1592, -0.1648, -0.0662, -0.3297, -0.9359],
        [-1.1842,  0.3562,  1.3115, -0.7578,  0.8986,  0.0

## Salvando resultados


In [13]:
from lib.plots import plot_density

values = [Adam_times, GA_times]
labels = ["Adam", "GA"]
plot_density(values, labels, filename="density")
